In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.dpi'] = 70 #display 70 dpi in Jupyter Notebook, may consider100 dpi 
plt.rcParams['savefig.dpi'] = 300 #define 300 dpi for saving figures

import seaborn as sns
## here are some settings 
sns.set_style('whitegrid')
sns.set(rc={"figure.dpi":70, 'savefig.dpi':300}) #defining dpi setting
sns.set_context('notebook')
sns.set_style("ticks")
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')

In [2]:
# Tells matplotlib to display images inline instead of a new window
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from time import time
import timeit #imports timeit module
start_time = timeit.default_timer() #defines start time so computational time can be calculated
print("Hello World")
elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time
print()

Hello World
---Run time is 0.0003207610000117711 seconds ---



In [13]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.preprocessing import PolynomialFeatures

import sklearn.linear_model as skl_lm
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn import neighbors
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis 
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.tree import DecisionTreeRegressor 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.tree import export_graphviz
from sklearn.svm import SVC

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import mean_squared_error

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from tqdm import tqdm
import wfdb
import numpy as np
import pandas as pd

In [34]:
# import os
# cwd = os.getcwd()
# print(cwd)
# os.chdir('/Users/chenc/000---STT450-Jupyter Notebook/000--STT450-550/') 
#data = load_breast_cancer(as_frame = True); 
#df = data.frame; 
#print(df.shape)
# In this block use the code we looked at on 6/30 as a reference to save our rr-intervals as a .
rlist = []
records = 'mit-bih-raw\RECORDS' # Replace the _______ with the name of the records file in your MIT-BIH data folder
with open(records) as rfile: #Then we open the file 
                             #The 'with' command only opens the file while we are in it. Automatically closes the file when we're not
    for record in rfile:  # Then we iterate through the lines in the file
        record = record[0:len(record)-1] # Remove any erronious new line characters at the end ('\n')
        rlist.append(record) # Then build an array with it
        

###### Step 1: Initialize all Arrays
             # Below, replace all of the ___ with the command that declares an array/list
             # hint: https://stackoverflow.com/questions/1514553/how-to-declare-an-array-in-python
samples = [] # will house the samples of all subjects
good_list = [] # will list the names of the subjects we successfully extracted
bad_list = [] # will house the names of the subjects we failed to extract
qrs = [] # will house the indices of R-Peaks for all subjects
atr_label = [] # will house the labels for each rhythm annotation for all subjects
atr_locs = [] # will house the locations corresponding to the rhythm annotation labels


###### Step 2: Extract Information
for x in tqdm(rlist): #this will iterate through te records that we found above
  
    try: # A try statement will run the except statement if for some reason the try commands fail
         # In this case I use the try statement because one of the subjects has no signal data causing failure
         # I then use bad_list and good_list so that all of the indices in rlist match with the arrays we initialized in Step 1, above
        ######################################################
            # Below find the wfdb function that will return the information that is described below 
        # Then replace _____ with the correct function call
        samp = wfdb.rdsamp('mit-bih-raw/'+x) # wfdb._____(file_location) will read the signal & header data and return a 2 value array
            # samp[0] - the signal data is the raw reading from the ecg. Each value is a sample taken.
            # samp[1] - the header data includes things about the signal data such as:
              # samples per section, denoted 'fs'
              # number of signals, denoted 'n_sig'
            
        ######################################################
        samples.append(samp) #add it to our array for all subject
            #What is our file extension that has the annotation we want? Find it here and replace _____ with it 
            #hint: READ THE VARIABLE NAMES!!!!
        qrs_tmp = wfdb.rdann('mit-bih-raw/'+x, extension='qrs') #extract the QRS Info
        qrs_locs = np.array(qrs_tmp.sample, dtype='int') #Get just the loccation of R-Peaks from the QRS Info
        qrs.append(qrs_locs) # Add to our array for all subjects
        
            #Do the same thing here
        atr = wfdb.rdann('mit-bih-raw/'+x,extension='atr') #extract the atr info which stores the rhythm type(s) over the whole signal
        atr_label.append(atr.aux_note) # aux_note stores the type of rhythm - main two are '(N' for normal and '(AFIB' for AFIB
        atr_locs.append(np.append(atr.sample, len(samp[0]))) #I add the length of the whole sample to the end for better visualization later
        
        good_list.append(x) # when all extraction is successful append the record name to good_list
    except Exception as exep:
        print(exep) # Alert the user of an exception
        bad_list.append(x) # add to the bad list

        

  0%|          | 0/25 [00:00<?, ?it/s]

sampto must be greater than sampfrom
sampto must be greater than sampfrom


100%|██████████| 25/25 [00:25<00:00,  1.02s/it]


In [36]:
# df3 = pd.read_csv('Data/Carseats.csv')
# #df3.info()
# print(); df3.head(5)
features_df_list = []
for subject in good_list:
    data = pd.read_csv(f'subject-features-dataframes/subject{subject}.csv')
    data = data.assign(Subject = subject)
    #print(data)
    features_df_list.append(data)
features_maindf = pd.concat(features_df_list, ignore_index = True)

In [ ]:
# df3['High'] = df3.Sales.map(lambda x: 1 if x>8 else 0)
# df3.ShelveLoc = pd.factorize(df3.ShelveLoc)[0]
# df3.Urban = df3.Urban.map({'No':0, 'Yes':1})
# df3.US = df3.US.map({'No':0, 'Yes':1})
# #df3.info()
# print(); df3.head(5)
#X = df3.drop(['Sales', 'High'], axis = 1)
#y = df3.High
##print(X.shape)
#print(y.shape)

In [47]:
x_values = ['StoS', 'StoR', 'StoL', 'RtoS', 'RtoR', 'RtoL', 'LtoS',
       'LtoR', 'LtoL', 'STD', 'CoefVar', 'Range', 'Volatility', 'ShEn',
       'AppEn', 'SampEn']
y_values = ['Rhythm']
df

,Unnamed: 0,StoS,StoR,StoL,RtoS,RtoR,RtoL,LtoS,LtoR,LtoL,STD,CoefVar,Range,Volatility,ShEn,AppEn,SampEn,Rhythm,Subject
0,0,0.250000,0.125000,0.083333,0.083333,0.208333,0.083333,0.083333,0.041667,0.041667,0.180045,0.259401,0.620,0.024543,1.050913,0.807078,1.004927,0,04015
1,1,0.208333,0.166667,0.041667,0.166667,0.291667,0.041667,0.041667,0.041667,0.000000,0.181063,0.314259,0.584,0.034714,0.915595,0.722627,0.867501,0,04015
2,2,0.083333,0.083333,0.166667,0.041667,0.375000,0.041667,0.166667,0.041667,0.000000,0.176833,0.224044,0.620,0.033321,1.038812,0.780945,0.871395,0,04015
3,3,0.166667,0.125000,0.000000,0.083333,0.416667,0.083333,0.041667,0.041667,0.041667,0.166312,0.240948,0.544,0.019996,0.917357,0.551398,0.559616,0,04015
4,4,0.291667,0.208333,0.041667,0.125000,0.000000,0.125000,0.166667,0.000000,0.041667,0.190016,0.289659,0.552,0.033423,0.989094,0.638499,0.850539,0,04015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44963,2360,0.291667,0.125000,0.083333,0.083333,0.125000,0.083333,0.083333,0.083333,0.041667,0.160217,0.265050,0.532,0.029727,1.038812,0.876587,1.116470,1,08455
44964,2361,0.250000,0.166667,0.041667,0.041667,0.166667,0.166667,0.125000,0.041667,0.000000,0.131288,0.230363,0.504,0.029151,1.050913,0.743082,0.842679,1,08455
44965,2362,0.250000,0.083333,0.041667,0.041667,0.375000,0.083333,0.083333,0.041667,0.000000,0.280151,0.453378,1.304,0.247255,0.973253,0.748702,0.796005,1,08455
44966,2363,0.166667,0.166667,0.083333,0.083333,0.208333,0.083333,0.166667,0.000000,0.041667,0.177207,0.311107,0.752,0.054405,1.050913,0.937331,1.217672,1,08455


### Decision Tree

criterion{“gini”, “entropy”}, default=”gini”

In [69]:
crossvalidation = KFold(n_splits=23)
#for df in features_df_list:
df = features_maindf
acc_score = [];
Truth = [];
Output = [];
for idx, subject in tqdm(enumerate(good_list)):
    
    train_index = ~df['Subject'].str.contains(subject)
    test_index = df['Subject'].str.contains(subject)
    X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
    y_train , y_test = df.loc[train_index,y_values].values.ravel(), df.loc[test_index,y_values]

    model = DecisionTreeClassifier(max_depth=6) #criterion='entropy'
    model.fit(X_train,y_train)

    pred_values = model.predict(X_test)
    pred_prob = model.predict_proba(X_test)[:, 1]
     
    acc = accuracy_score(y_test, pred_values)
    acc_score.append(acc)
    
    Truth.extend(y_test.values.reshape(y_test.shape[0])); ## it is a list
    Output.extend(pred_values); ## it is a list 

23it [00:11,  2.03it/s]


In [70]:
print(acc_score)
print(confusion_matrix(Truth, Output))
print(classification_report(Truth, Output))
print(confusion_matrix_df)

[0.9073226544622426, 0.9456389452332657, 0.9955919395465995, 0.8872901678657075, 0.9926854754440961, 0.954158215010142, 0.8346567024754787, 0.9979577944179714, 0.929718875502008, 0.958103638368247, 0.9773139745916516, 0.9885139985642498, 0.7868480725623582, 0.943312101910828, 0.5974188176519567, 0.9676561807709348, 0.9862919808087731, 0.9826889786497404, 0.6225697379543533, 0.9120395821880154, 0.9901625320786998, 0.992467043314501, 0.9839323467230444]
[[22502  2241]
 [ 1791 18434]]
              precision    recall  f1-score   support

           0       0.93      0.91      0.92     24743
           1       0.89      0.91      0.90     20225

    accuracy                           0.91     44968
   macro avg       0.91      0.91      0.91     44968
weighted avg       0.91      0.91      0.91     44968

Empty DataFrame
Columns: [TN, FP, FN, TP]
Index: []


### Random Forests: use sqrt(p) for classification

In [67]:
crossvalidation = KFold(n_splits=23)
#for df in features_df_list:
df = features_maindf
acc_score = [];
Truth = [];
Output = [];
#confusion_matrix_df = pd.DataFrame(columns=['TN', 'FP', 'FN', 'TP'])
for idx, subject in tqdm(enumerate(good_list)):
    
    train_index = ~df['Subject'].str.contains(subject)
    test_index = df['Subject'].str.contains(subject)
    X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
    y_train , y_test = df.loc[train_index,y_values].values.ravel(), df.loc[test_index,y_values]

    model = RandomForestClassifier(max_features = 10, random_state = 2)
    model.fit(X_train,y_train)

    pred_values = model.predict(X_test)
    pred_prob = model.predict_proba(X_test)[:, 1]
     
    acc = accuracy_score(y_test, pred_values)
    acc_score.append(acc)
    #confusion_matrix_df.loc[idx] = confusion_matrix(Truth, Output).ravel()
    Truth.extend(y_test.values.reshape(y_test.shape[0])); ## it is a list
    Output.extend(pred_values); ## it is a list 

23it [10:58, 28.64s/it]


In [68]:
print(acc_score)
print(confusion_matrix(Truth, Output))
print(classification_report(Truth, Output))
print(confusion_matrix_df)

[0.9147597254004577, 0.9521298174442191, 0.9955919395465995, 0.9268585131894485, 0.9968652037617555, 0.954158215010142, 0.7930873423633816, 0.9979577944179714, 0.9407630522088354, 0.9707828004410143, 0.9823049001814882, 0.9899497487437185, 0.8267573696145125, 0.9490445859872612, 0.6032472939217319, 0.9867080194949047, 0.9883481836874571, 0.9861511829197923, 0.7523245984784447, 0.9708631115997801, 0.9931565440547476, 0.9949780288763339, 0.9953488372093023]
[[23105  1638]
 [ 1707 18518]]
              precision    recall  f1-score   support

           0       0.93      0.93      0.93     24743
           1       0.92      0.92      0.92     20225

    accuracy                           0.93     44968
   macro avg       0.92      0.92      0.92     44968
weighted avg       0.93      0.93      0.93     44968

Empty DataFrame
Columns: [TN, FP, FN, TP]
Index: []


In [72]:
#help(confusion_matrix)

In [ ]:
# crossvalidation = KFold(n_splits=23)
# #for df in features_df_list:
# df = features_maindf
# acc_score = [];
# Truth = [];
# Output = [];
# for idx, subject in tqdm(enumerate(good_list)):
    
#     train_index = ~df['Subject'].str.contains(subject)
#     test_index = df['Subject'].str.contains(subject)
#     X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
#     y_train , y_test = df.loc[train_index,y_values], df.loc[test_index,y_values]

#     model = RandomForestClassifier(max_features = 10, random_state = 2)
#     model.fit(X_train,y_train.values.ravel())

#     pred_values = model.predict(X_test)
#     pred_prob = model.predict_proba(X_test)[:, 1]
     
#     acc = accuracy_score(y_test, pred_values)
#     acc_score.append(acc)
    
#     Truth.extend(y_test.values.reshape(y_test.shape[0])); ## it is a list
#     Output.extend(pred_values); ## it is a list 


# model = RandomForestClassifier(max_features = 16, random_state = 2)
# scores = cross_val_score(model, X, y, cv=crossvalidation)
# print('Accuracy of each fold: \n {}'.format(scores))
# print()
# print("Avg accuracy: {}".format(scores.mean()))

### An AdaBoost classifier.

The argument `n_estimators = 500` indicates that we want 500 trees

class sklearn.ensemble.AdaBoostClassifier(base_estimator=None, *, n_estimators=50, learning_rate=1.0, algorithm='SAMME.R', random_state=None)

In [71]:
crossvalidation = KFold(n_splits=23)
#for df in features_df_list:
df = features_maindf
acc_score = [];
Truth = [];
Output = [];
for idx, subject in tqdm(enumerate(good_list)):
    
    train_index = ~df['Subject'].str.contains(subject)
    test_index = df['Subject'].str.contains(subject)
    X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
    y_train , y_test = df.loc[train_index,y_values].values.ravel(), df.loc[test_index,y_values]

    model = AdaBoostClassifier(n_estimators=500, learning_rate = 0.1, algorithm="SAMME.R", random_state=2)
    model.fit(X_train,y_train)

    pred_values = model.predict(X_test)
    pred_prob = model.predict_proba(X_test)[:, 1]
     
    acc = accuracy_score(y_test, pred_values)
    acc_score.append(acc)
    
    Truth.extend(y_test.values.reshape(y_test.shape[0])); ## it is a list
    Output.extend(pred_values); ## it is a list 

23it [16:11, 42.24s/it]


In [73]:
print(acc_score)
print(confusion_matrix(Truth, Output))
print(classification_report(Truth, Output))
print(confusion_matrix_df)

[0.9090389016018307, 0.9521298174442191, 0.9949622166246851, 0.9274580335731415, 0.9958202716823407, 0.9464503042596348, 0.8267164876226063, 0.9972770592239619, 0.9437751004016064, 0.9735391400220507, 0.98502722323049, 0.9892318736539842, 0.7419501133786848, 0.9560509554140127, 0.5628642797668609, 0.9875941515285778, 0.9890335846470185, 0.9803808424697057, 0.6910397295012679, 0.9785596481583287, 0.9961505560307955, 0.9968612680477087, 0.9957716701902748]
[[22760  1983]
 [ 1706 18519]]
              precision    recall  f1-score   support

           0       0.93      0.92      0.93     24743
           1       0.90      0.92      0.91     20225

    accuracy                           0.92     44968
   macro avg       0.92      0.92      0.92     44968
weighted avg       0.92      0.92      0.92     44968

Empty DataFrame
Columns: [TN, FP, FN, TP]
Index: []


### Gradient Boosting for classification

In [75]:
crossvalidation = KFold(n_splits=23)
#for df in features_df_list:
df = features_maindf
acc_score = [];
Truth = [];
Output = [];
#confusion_matrix_df = pd.DataFrame(columns=['TN', 'FP', 'FN', 'TP'])
for idx, subject in tqdm(enumerate(good_list)):
    
    train_index = ~df['Subject'].str.contains(subject)
    test_index = df['Subject'].str.contains(subject)
    X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
    y_train , y_test = df.loc[train_index,y_values].values.ravel(), df.loc[test_index,y_values]

    model =  GradientBoostingClassifier(n_estimators = 500, 
                                           learning_rate = 0.1, 
                                           max_depth = 4, 
                                           random_state = 2)
    model.fit(X_train,y_train)

    pred_values = model.predict(X_test)
    pred_prob = model.predict_proba(X_test)[:, 1]
     
    acc = accuracy_score(y_test, pred_values)
    acc_score.append(acc)
    #confusion_matrix_df.loc[idx] = confusion_matrix(Truth, Output).ravel()
    Truth.extend(y_test.values.reshape(y_test.shape[0])); ## it is a list
    Output.extend(pred_values); ## it is a list 

23it [44:20, 115.69s/it]


In [76]:
print(acc_score)
print(confusion_matrix(Truth, Output))
print(classification_report(Truth, Output))
print(confusion_matrix_df)

[0.9136155606407322, 0.9565922920892495, 0.9955919395465995, 0.9298561151079137, 0.9968652037617555, 0.9594320486815415, 0.7603923400280242, 0.998638529611981, 0.9397590361445783, 0.9680264608599779, 0.9832123411978222, 0.9899497487437185, 0.8444444444444444, 0.9464968152866242, 0.6149042464612823, 0.9884802835622508, 0.9883481836874571, 0.9849971148297749, 0.7603550295857988, 0.9494227597581089, 0.9948674080410608, 0.9962335216572504, 0.9949260042283298]
[[23139  1604]
 [ 1738 18487]]
              precision    recall  f1-score   support

           0       0.93      0.94      0.93     24743
           1       0.92      0.91      0.92     20225

    accuracy                           0.93     44968
   macro avg       0.93      0.92      0.92     44968
weighted avg       0.93      0.93      0.93     44968

Empty DataFrame
Columns: [TN, FP, FN, TP]
Index: []
